In [12]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# Define the range of times from 00 to 12
times = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

# Given dates
dates = ["2022-02-24T00:00:00.000000", "2022-02-28T00:00:00.000000", "2022-03-08T00:00:00.000000",
         "2022-03-16T00:00:00.000000", "2022-03-22T00:00:00.000000", "2022-04-05T00:00:00.000000",
         "2022-04-12T00:00:00.000000", "2022-04-20T00:00:00.000000", "2022-04-29T00:00:00.000000",
         "2022-05-03T00:00:00.000000", "2022-05-11T00:00:00.000000", "2022-05-17T00:00:00.000000",
         "2022-05-29T00:00:00.000000"]

# Extract dates without times
dates_without_times = [date.split('T')[0] for date in dates]

# Loop over times
for time in times:

    # Informative message: Loading datasets for the current time
    print(f'Processing data for time: {time}')
    
    # Load chl and PFT datasets for the current time
    chl_ds = xr.open_dataset(f'chl_aviris_dangermond_time_{time}_reg.nc')
    lai_ds = xr.open_dataset(f'lai_aviris_dangermond_time_{time}_reg.nc')
    lma_ds = xr.open_dataset(f'lma_aviris_dangermond_time_{time}_reg.nc')
    pft_ds = xr.open_dataset(f'../California_Vegetation_WHRTYPE_Dangermond/output_latlon.nc')  # Update the PFT dataset filename
    
    # Informative message: Extracting data from the loaded datasets
    print('Extracting data from the loaded datasets...')
    
    # Extract chl values and PFT data
    chl_values = chl_ds['chl'].values
    lai_values = lai_ds['lai'].values
    lma_values = lma_ds['lma'].values
    pft_values = pft_ds['Band1'].values  # Assuming PFT values are stored in Band1 variable
    
    # Define PFT categories (2, 3, 4)
    pft_categories = [2, 3, 4]
    
    # Create a mask for PFT categories
    pft_mask = np.isin(pft_values, pft_categories)
    
    # Ensure chl_values and pft_mask have the same shape
    masked_chl_values = np.ma.masked_where(~pft_mask[:, :], chl_values[0, :, :])
    masked_lai_values = np.ma.masked_where(~pft_mask[:, :], lai_values[0, :, :])
    masked_lma_values = np.ma.masked_where(~pft_mask[:, :], lma_values[0, :, :])
    
    # Calculate average chl value per PFT
    average_chl_per_pft = []
    std_chl_per_pft = []
    average_lai_per_pft = []
    std_lai_per_pft = []
    average_lma_per_pft = []
    std_lma_per_pft = []
    for pft_category in pft_categories:
        # chl
        pft_chl_values = masked_chl_values[pft_values[:, :] == pft_category]
        average_chl = np.nanmean(pft_chl_values)
        std_chl = np.nanstd(pft_chl_values)
        average_chl_per_pft.append(average_chl)
        std_chl_per_pft.append(std_chl)
        
        # lai
        pft_lai_values = masked_lai_values[pft_values[:, :] == pft_category]
        average_lai = np.nanmean(pft_lai_values)
        std_lai = np.nanstd(pft_lai_values)
        average_lai_per_pft.append(average_lai)
        std_lai_per_pft.append(std_lai)
        
        # lma
        pft_lma_values = masked_lma_values[pft_values[:, :] == pft_category]
        pft_lma_values = pft_lma_values[(~np.isnan(pft_lma_values)) & (pft_lma_values != 0)]
        average_lma = np.nanmean(pft_lma_values)
        std_lma = np.nanstd(pft_lma_values)
        average_lma_per_pft.append(average_lma)
        std_lma_per_pft.append(std_lma)
    
    # Print average and standard deviation chl values per PFT
    for idx, (avg_chl, std_chl) in enumerate(zip(average_chl_per_pft, std_chl_per_pft)):
        print(f'PFT {pft_categories[idx] -1 }: Average chl value: {avg_chl}, Standard Deviation: {std_chl}')

    # Print average and standard deviation lai values per PFT
    for idx, (avg_lai, std_lai) in enumerate(zip(average_lai_per_pft, std_lai_per_pft)):
        print(f'PFT {pft_categories[idx] -1 }: Average lai value: {avg_lai}, Standard Deviation: {std_lai}')

    # Print average and standard deviation lma values per PFT
    for idx, (avg_lma, std_lma) in enumerate(zip(average_lma_per_pft, std_lma_per_pft)):
        print(f'PFT {pft_categories[idx]-1}: Average lma value: {avg_lma}, Standard Deviation: {std_lma}')

    # Optionally, save the new chl map with masked values
    # chl_ds['chl'][:] = masked_chl_values
    # chl_ds.to_netcdf('masked_chl_aviris_dangermond.nc')

    # Create a new chl map with average values per PFT
    new_chl_values = np.zeros_like(chl_values[0,:,:])
    new_std_chl_values = np.zeros_like(chl_values[0,:,:])
    new_lai_values = np.zeros_like(lai_values[0,:,:])
    new_std_lai_values = np.zeros_like(lai_values[0,:,:])
    new_lma_values = np.zeros_like(lma_values[0,:,:])
    new_std_lma_values = np.zeros_like(lma_values[0,:,:])


    # Creating plots for all PFTs for chl
    #plt.figure()
    plt.figure(figsize=(8, 6))  # Adjust figure size
    colors = ['g', 'b', 'r']  # Colors for PFTs 2, 3, 4
    legend_labels = []

    for idx, pft_category in enumerate(pft_categories):
        mask = (pft_values == pft_category)[:, :]
        new_chl_values[mask] = average_chl_per_pft[idx]
        new_std_chl_values[mask] = std_chl_per_pft[idx]
        #new_lai_values[mask] = average_lai_per_pft[idx]
        #new_std_lai_values[mask] = std_lai_per_pft[idx]
        #new_lma_values[mask] = average_lma_per_pft[idx]
        #new_std_lma_values[mask] = std_lma_per_pft[idx]

        # Extract chl values for the current PFT category
        chl_for_pft = masked_chl_values[pft_values == pft_category].compressed()
        avg_chl = average_chl_per_pft[idx]
        std_chl = std_chl_per_pft[idx]

        # Create a histogram for chl values with transparency
        plt.hist(chl_for_pft, bins=30, density=True, alpha=0.5, color=colors[idx])

        # Add a vertical line for the average
        plt.axvline(avg_chl, color=colors[idx], linestyle='dashed', linewidth=1)

        # Prepare text for the legend
        avg_std_text = f'{avg_chl:.2f} ± {std_chl:.2f}'
        legend_labels.append(f'PFT {pft_category}: {avg_std_text}')

    plt.title(f'Combined CHL Distribution ({dates_without_times[int(time)]})', fontsize=18)
    plt.xlabel(r'CHL Value ($\mu$g.cm$^{-2}$)', fontsize=16)
    plt.ylabel('Density', fontsize=16)
    plt.ylim(0, 0.5)  # Set y-axis limits
    plt.xlim(0, 20)  # Set y-axis limits
    plt.legend(legend_labels, fontsize=14)  # Increase legend fontsize
    plt.xticks(fontsize=14)  # Increase x-axis tick label fontsize
    plt.yticks(fontsize=14)  # Increase y-axis tick label fontsize

    # Save the combined plot as a PNG file
    plt.savefig(f'combined_chl_histogram_time_{time}.png',dpi=300)
    plt.close()

    # Creating plots for all PFTs for lai
    plt.figure(figsize=(8, 6))  # Adjust figure size
    colors = ['g', 'b', 'r']  # Colors for PFTs 2, 3, 4
    legend_labels = []

    for idx, pft_category in enumerate(pft_categories):
        mask = (pft_values == pft_category)[:, :]
        #new_chl_values[mask] = average_chl_per_pft[idx]
        #new_std_chl_values[mask] = std_chl_per_pft[idx]
        new_lai_values[mask] = average_lai_per_pft[idx]
        new_std_lai_values[mask] = std_lai_per_pft[idx]
        #new_lma_values[mask] = average_lma_per_pft[idx]
        #new_std_lma_values[mask] = std_lma_per_pft[idx]

        # Extract chl values for the current PFT category
        lai_for_pft = masked_lai_values[pft_values == pft_category].compressed()
        avg_lai = average_lai_per_pft[idx]
        std_lai = std_lai_per_pft[idx]

        # Create a histogram for chl values with transparency
        plt.hist(lai_for_pft, bins=30, density=True, alpha=0.5, color=colors[idx])

        # Add a vertical line for the average
        plt.axvline(avg_lai, color=colors[idx], linestyle='dashed', linewidth=1)

        # Prepare text for the legend
        avg_std_text = f'{avg_lai:.2f} ± {std_lai:.2f}'
        legend_labels.append(f'PFT {pft_category}: {avg_std_text}')

    plt.title(f'Combined LAI Distribution ({dates_without_times[int(time)]})', fontsize=18)
    plt.xlabel(r'LAI Value (m$^{2}$.m$^{-2}$)', fontsize=16)
    plt.ylabel('Density', fontsize=16)
    plt.ylim(0, 1.0)  # Set y-axis limits
    plt.xlim(0, 7.5)  # Set y-axis limits
    plt.legend(legend_labels, fontsize=14)  # Increase legend fontsize
    plt.xticks(fontsize=14)  # Increase x-axis tick label fontsize
    plt.yticks(fontsize=14)  # Increase y-axis tick label fontsize

    # Save the combined plot as a PNG file
    plt.savefig(f'combined_lai_histogram_time_{time}.png')
    plt.close()

    # Creating plots for all PFTs for lma
    plt.figure(figsize=(8, 6))  # Adjust figure size
    colors = ['g', 'b', 'r']  # Colors for PFTs 2, 3, 4
    legend_labels = []

    for idx, pft_category in enumerate(pft_categories):
        mask = (pft_values == pft_category)[:, :]
        #new_chl_values[mask] = average_chl_per_pft[idx]
        #new_std_chl_values[mask] = std_chl_per_pft[idx]
        #new_lai_values[mask] = average_lai_per_pft[idx]
        #new_std_lai_values[mask] = std_lai_per_pft[idx]
        new_lma_values[mask] = average_lma_per_pft[idx]
        new_std_lma_values[mask] = std_lma_per_pft[idx]

        # Extract chl values for the current PFT category
        lma_for_pft = masked_lma_values[pft_values == pft_category].compressed()
        avg_lma = average_lma_per_pft[idx]
        std_lma = std_lma_per_pft[idx]

        # Create a histogram for chl values with transparency
        plt.hist(lma_for_pft, bins=30, density=True, alpha=0.5, color=colors[idx])

        # Add a vertical line for the average
        plt.axvline(avg_lma, color=colors[idx], linestyle='dashed', linewidth=1)

        # Prepare text for the legend
        avg_std_text = f'{avg_lma:.2e} ± {std_lma:.2e}'
        legend_labels.append(f'PFT {pft_category}: {avg_std_text}')

    print(np.min(lma_for_pft),np.max(lma_for_pft))
    plt.title(f'Combined LMA Distribution ({dates_without_times[int(time)]})', fontsize=18)
    plt.xlabel(r'LMA Value (g.cm$^{-2}$)', fontsize=16)
    plt.ylabel('Density', fontsize=16)
    plt.ylim(0, 800)  # Set y-axis limits
    plt.xlim(0, 0.03)  # Set y-axis limits
    plt.legend(legend_labels, fontsize=14)  # Increase legend fontsize
    plt.xticks(fontsize=14)  # Increase x-axis tick label fontsize
    plt.yticks(fontsize=14)  # Increase y-axis tick label fontsize

    # Save the combined plot as a PNG file
    plt.savefig(f'combined_lma_histogram_time_{time}.png')
    plt.close()

    

    # Create new xarray DataArrays for chl and std
    new_chl_da = xr.DataArray(new_chl_values[np.newaxis,:,:], dims=('time', 'lat', 'lon'), coords={'time': chl_ds['time'], 'lat': chl_ds['lat'], 'lon': chl_ds['lon']})
    new_std_chl_da = xr.DataArray(new_std_chl_values[np.newaxis,:,:], dims=('time', 'lat', 'lon'), coords={'time': chl_ds['time'], 'lat': chl_ds['lat'], 'lon': chl_ds['lon']})


    # Replace zeros by NaN in new maps
    new_chl_da = new_chl_da.where(new_chl_da != 0, np.nan)
    new_std_chl_da = new_std_chl_da.where(new_std_chl_da != 0, np.nan)

    # Create a new xarray Dataset with chl and std DataArrays
    new_chl_data = {'chl': new_chl_da, 'std': new_std_chl_da}
    new_chl_ds = xr.Dataset(new_chl_data)
    

    # Save the new chl map with masked values and standard deviation to a NetCDF file
    new_chl_ds.to_netcdf(f'masked_chl_aviris_dangermond_time_{time}.nc')

    print(lai_values.shape)
    print(new_lai_values.shape)
    # Create new xarray DataArrays for lai and std
    #new_lai_da = xr.DataArray(new_lai_values[np.newaxis,:,:], dims=('time', 'lat', 'lon'), coords={'time': lai_ds['time'], 'lat': lai_ds['lat'], 'lon': lai_ds['lon']})
    lai_values = np.clip(lai_values, 1e-1, 20.)  # Clip values between 0.1 and max_lai
    new_lai_values = lai_values[0,:,:]*(new_std_lai_values/new_std_lai_values)
    
    new_lai_da = xr.DataArray(new_lai_values[np.newaxis,:,:], dims=('time', 'lat', 'lon'), coords={'time': lai_ds['time'], 'lat': lai_ds['lat'], 'lon': lai_ds['lon']})
    new_std_lai_da = xr.DataArray(new_std_lai_values[np.newaxis,:,:], dims=('time', 'lat', 'lon'), coords={'time': lai_ds['time'], 'lat': lai_ds['lat'], 'lon': lai_ds['lon']})
    
    # Replace zeros by NaN in new maps
    new_lai_da = new_lai_da.where(new_lai_da != 0, np.nan)
    new_lai_da = new_lai_da.where(~np.isnan(new_lai_da), np.nan)

    new_std_lai_da = new_std_lai_da.where(new_std_lai_da != 0, np.nan)

    # Create a new xarray Dataset with lai and std DataArrays
    new_lai_data = {'lai': new_lai_da, 'std': new_std_lai_da}
    new_lai_ds = xr.Dataset(new_lai_data)
    
    # Save the new lai map with masked values and standard deviation to a NetCDF file
    new_lai_ds.to_netcdf(f'masked_lai_aviris_dangermond_time_{time}_v1.nc')

    # Create new xarray DataArrays for lma and std
    new_lma_da = xr.DataArray(new_lma_values[np.newaxis,:,:], dims=('time', 'lat', 'lon'), coords={'time': lma_ds['time'], 'lat': lma_ds['lat'], 'lon': lma_ds['lon']})
    new_std_lma_da = xr.DataArray(new_std_lma_values[np.newaxis,:,:], dims=('time', 'lat', 'lon'), coords={'time': lma_ds['time'], 'lat': lma_ds['lat'], 'lon': lma_ds['lon']})
    
    new_lma_da = new_lma_da.where(new_lma_da != 0, np.nan)
    new_std_lma_da = new_std_lma_da.where(new_std_lma_da != 0, np.nan)

    # Create a new xarray Dataset with lma and std DataArrays
    new_lma_data = {'lma': new_lma_da, 'std': new_std_lma_da}
    new_lma_ds = xr.Dataset(new_lma_data)
    


    # Save the new lma map with masked values and standard deviation to a NetCDF file
    new_lma_ds.to_netcdf(f'masked_lma_aviris_dangermond_time_{time}.nc')
    
    
    # Informative message: Data processing and saving completed for the current time
    print(f'Data processing and saving completed for time: {time}\n')

    # Close the opened datasets
    chl_ds.close()
    lai_ds.close()
    lma_ds.close()
    pft_ds.close()



Processing data for time: 00
Extracting data from the loaded datasets...
PFT 1: Average chl value: 8.101466178894043, Standard Deviation: 2.7580385208129883
PFT 2: Average chl value: 8.212648391723633, Standard Deviation: 2.7896728515625
PFT 3: Average chl value: 9.34757137298584, Standard Deviation: 2.72875714302063
PFT 1: Average lai value: 1.7422716617584229, Standard Deviation: 0.8995611071586609
PFT 2: Average lai value: 1.9047859907150269, Standard Deviation: 0.9417217373847961
PFT 3: Average lai value: 2.1260483264923096, Standard Deviation: 0.7934799194335938
PFT 1: Average lma value: 0.010749610140919685, Standard Deviation: 0.008235991932451725
PFT 2: Average lma value: 0.009704625234007835, Standard Deviation: 0.007617283146828413
PFT 3: Average lma value: 0.014527236111462116, Standard Deviation: 0.008670768700540066
nan nan
(1, 458, 492)
(458, 492)
Data processing and saving completed for time: 00

Processing data for time: 01
Extracting data from the loaded datasets...
PF

/tmp/ipykernel_1689607/3132838646.py:258: RuntimeWarning: invalid value encountered in divide
  new_lai_values = lai_values[0,:,:]*(new_std_lai_values/new_std_lai_values)


nan nan
(1, 458, 492)
(458, 492)


/tmp/ipykernel_1689607/3132838646.py:258: RuntimeWarning: invalid value encountered in divide
  new_lai_values = lai_values[0,:,:]*(new_std_lai_values/new_std_lai_values)


Data processing and saving completed for time: 01

Processing data for time: 02
Extracting data from the loaded datasets...
PFT 1: Average chl value: 8.050215721130371, Standard Deviation: 2.858710289001465
PFT 2: Average chl value: 8.080536842346191, Standard Deviation: 2.8197522163391113
PFT 3: Average chl value: 9.482297897338867, Standard Deviation: 2.7919137477874756
PFT 1: Average lai value: 1.6817245483398438, Standard Deviation: 0.8799581527709961
PFT 2: Average lai value: 1.7683489322662354, Standard Deviation: 0.8583741784095764
PFT 3: Average lai value: 2.154921531677246, Standard Deviation: 0.813147246837616
PFT 1: Average lma value: 0.010249734856188297, Standard Deviation: 0.008123805746436119
PFT 2: Average lma value: 0.009185192175209522, Standard Deviation: 0.0074811032973229885
PFT 3: Average lma value: 0.013825262896716595, Standard Deviation: 0.008526615798473358
nan nan
(1, 458, 492)
(458, 492)


...

/tmp/ipykernel_1689607/3132838646.py:258: RuntimeWarning: invalid value encountered in divide
  new_lai_values = lai_values[0,:,:]*(new_std_lai_values/new_std_lai_values)


Data processing and saving completed for time: 10

Processing data for time: 11
Extracting data from the loaded datasets...
PFT 1: Average chl value: 4.331429481506348, Standard Deviation: 2.6441659927368164
PFT 2: Average chl value: 3.8491060733795166, Standard Deviation: 2.499204158782959
PFT 3: Average chl value: 6.684986114501953, Standard Deviation: 3.0851967334747314
PFT 1: Average lai value: 0.9166612029075623, Standard Deviation: 0.5507084131240845
PFT 2: Average lai value: 0.8715827465057373, Standard Deviation: 0.49058422446250916
PFT 3: Average lai value: 1.3235414028167725, Standard Deviation: 0.7195901274681091
PFT 1: Average lma value: 0.00774308480322361, Standard Deviation: 0.007180014159530401
PFT 2: Average lma value: 0.007206910289824009, Standard Deviation: 0.00672659557312727
PFT 3: Average lma value: 0.010871895588934422, Standard Deviation: 0.008205029182136059
nan nan
(1, 458, 492)
(458, 492)


/tmp/ipykernel_1689607/3132838646.py:258: RuntimeWarning: invalid value encountered in divide
  new_lai_values = lai_values[0,:,:]*(new_std_lai_values/new_std_lai_values)


Data processing and saving completed for time: 11

Processing data for time: 12
Extracting data from the loaded datasets...
PFT 1: Average chl value: 4.313188552856445, Standard Deviation: 2.7828786373138428
PFT 2: Average chl value: 3.7588918209075928, Standard Deviation: 2.6270782947540283
PFT 3: Average chl value: 6.7374749183654785, Standard Deviation: 3.1690433025360107
PFT 1: Average lai value: 0.9502406716346741, Standard Deviation: 0.5810040831565857
PFT 2: Average lai value: 0.8658716678619385, Standard Deviation: 0.5175431370735168
PFT 3: Average lai value: 1.4681825637817383, Standard Deviation: 0.7003843784332275
PFT 1: Average lma value: 0.007621224503964186, Standard Deviation: 0.007352003827691078
PFT 2: Average lma value: 0.00673070689663291, Standard Deviation: 0.00668569840490818
PFT 3: Average lma value: 0.012145447544753551, Standard Deviation: 0.008403096348047256
nan nan
(1, 458, 492)
(458, 492)
Data processing and saving completed for time: 12



/tmp/ipykernel_1689607/3132838646.py:258: RuntimeWarning: invalid value encountered in divide
  new_lai_values = lai_values[0,:,:]*(new_std_lai_values/new_std_lai_values)


In [4]:
from PIL import Image
import os

# Directory containing the PNG images
image_directory = '/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/'

# Output GIF file name
output_gif = 'combined_lma_histogram.gif'

# List of PNG files in the directory
png_files = [f for f in os.listdir(image_directory) if f.startswith('combined_lma_histogram_time_') and f.endswith('.png')]
png_files.sort()  # Sort the files in ascending order by filename

# Create a list to store image frames
frames = []

# Open and append each PNG image to the frames list
for png_file in png_files:
    image_path = os.path.join(image_directory, png_file)
    img = Image.open(image_path)
    frames.append(img)

# Save the frames as an animated GIF
frames[0].save(output_gif, save_all=True, append_images=frames[1:], duration=500, loop=0)

print(f'GIF saved as {output_gif}')


GIF saved as combined_lma_histogram.gif


In [11]:
from PIL import Image
import os

# Directory containing the PNG images
image_directory = '/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/'

# Output GIF file name
output_gif = 'combined_chl_histogram.gif'

# List of PNG files in the directory
png_files = [f for f in os.listdir(image_directory) if f.startswith('combined_chl_histogram_time_') and f.endswith('.png')]
png_files.sort()  # Sort the files in ascending order by filename

# Create a list to store image frames
frames = []

# Open and append each PNG image to the frames list
for png_file in png_files:
    image_path = os.path.join(image_directory, png_file)
    img = Image.open(image_path)
    frames.append(img)

# Save the frames as an animated GIF
frames[0].save(output_gif, save_all=True, append_images=frames[1:], duration=500, loop=0)

print(f'GIF saved as {output_gif}')


GIF saved as combined_chl_histogram.gif


In [10]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

# Define the range of times from 00 to 12
times = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

# Define PFT categories
pft_categories = [2, 3, 4]
#pft_categories = [1, 2, 3]
colors = ['g', 'b', 'r']  # Colors for PFTs 2, 3, 4

# Initialize accumulators for each trait and PFT
data_accumulator = {trait: {pft: [] for pft in pft_categories} for trait in ['chl', 'lai', 'lma']}

# Loop over times to aggregate data
for time in times:
    print(f'Processing data for time: {time}')

    # Load datasets for the current time
    chl_ds = xr.open_dataset(f'chl_aviris_dangermond_time_{time}_reg.nc')
    lai_ds = xr.open_dataset(f'lai_aviris_dangermond_time_{time}_reg.nc')
    lma_ds = xr.open_dataset(f'lma_aviris_dangermond_time_{time}_reg.nc')
    pft_ds = xr.open_dataset(f'../California_Vegetation_WHRTYPE_Dangermond/output_latlon.nc')

    # Extract data
    chl_values = chl_ds['chl'].values.flatten()
    lai_values = lai_ds['lai'].values.flatten()
    #change units from g.cm-2 to g.m-2
    lma_values = lma_ds['lma'].values.flatten()*100*100
    pft_values = pft_ds['Band1'].values.flatten()

    # Aggregate data for each PFT
    for pft in pft_categories:
        pft_mask = pft_values == pft
        data_accumulator['chl'][pft].extend(chl_values[pft_mask])
        data_accumulator['lai'][pft].extend(lai_values[pft_mask])
        data_accumulator['lma'][pft].extend(lma_values[pft_mask])

# Function to plot time-averaged histogram per PFT for a given trait
def plot_time_averaged_histogram(data_accumulator, trait, colors, pft_categories):
    plt.figure(figsize=(8, 6))
    legend_labels = []

    for idx, pft in enumerate(pft_categories):
        data = np.array(data_accumulator[trait][pft])
        valid_data = data[~np.isnan(data)]  # Remove NaNs

        # Exclude the extreme values
        min_val = np.nanmin(valid_data)
        max_val = np.nanmax(valid_data)
        data_to_plot = valid_data[(valid_data > min_val) & (valid_data < max_val)]

        # Calculate average and std
        avg, std = np.nanmean(data_to_plot), np.nanstd(data_to_plot)

        # Plot histogram with a log scale if it's LMA
        if trait == 'lma':
            plt.yscale('log')  # Set y-axis to log scale for LMA

        plt.hist(data_to_plot, bins=30, color=colors[idx], alpha=0.5, density=True)
        plt.axvline(avg, color=colors[idx], linestyle='dashed', linewidth=1)

        # Legend without units
        legend_labels.append(f'PFT {pft-1}: {avg:.2f} ± {std:.2f}' if trait == 'lma' else f'PFT {pft-1}: {avg:.2f} ± {std:.2f}')

    # Set units for x-axis label
    unit = 'μg/cm²' if trait == 'chl' else ('m²/m²' if trait == 'lai' else 'g/m²')
    plt.title(f'Time-Averaged {trait.upper()} Distribution by PFT', fontsize=18)
    plt.xlabel(f'{trait.upper()} Value ({unit})', fontsize=16)
    plt.ylabel('Density', fontsize=16)
    plt.legend(legend_labels, fontsize=14)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.savefig(f'time_averaged_{trait}_distribution_by_pft.png')
    plt.close()

# Plot for each trait
for trait in ['chl', 'lai', 'lma']:
    plot_time_averaged_histogram(data_accumulator, trait, colors, pft_categories)


Processing data for time: 00
Processing data for time: 01
Processing data for time: 02
Processing data for time: 03
Processing data for time: 04
Processing data for time: 05
Processing data for time: 06
Processing data for time: 07
Processing data for time: 08
Processing data for time: 09
Processing data for time: 10
Processing data for time: 11
Processing data for time: 12
